In [21]:
from dotenv import load_dotenv
import logging
import os

from config import endpoints, card, user, store, order, product, date_times, DataType
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
from database_utils import DatabaseConnector

logging.basicConfig(filename='pipeline.log', encoding='utf-8', level=logging.DEBUG,
                    format="%(asctime)s [%(levelname)s] %(name)s - %(funcName).40s - %(message)s",)
logger = logging.getLogger(__name__)
load_dotenv()


def setup_database(filename):
    db_conn = DatabaseConnector()
    db_credentials = db_conn.read_db_creds(filename)
    engine = db_conn.init_db_engine(db_credentials)
    return db_conn, engine


ImportError: cannot import name 'endpoints' from 'config' (/Users/mashie/src/multinational-retail-data-centralisation/config.py)

In [13]:
source_db, source_engine = setup_database(filename='config/db_creds.yaml')
target_db, target_engine = setup_database(filename='config/db_creds_target.yaml')

print(f"Processing {data_type.name} Data")
headers = {
    "Content-Type": "application/json",
    "X-API-KEY": os.getenv('x-api-key')
}

# Extract
data_extractor = DataExtractor()
num_stores = data_extractor.list_number_of_stores(url=endpoints.number_of_stores, headers=headers)
df_extracted = data_extractor.retrieve_stores_data(
    url=endpoints.store_details, headers=headers, number_stores=num_stores)
print(f"{data_type.name} rows extracted: {len(df_extracted.index)}")
assert len(df_extracted.index) == data_type.extracted_count

# Clean
data_cleaner = DataCleaning(column_entries=data_type.column_entries)
df_cleaned = df_extracted.copy()
df_cleaned = data_cleaner.clean_store_data(df=df_cleaned)
print(f"{data_type.name} rows after cleaning: {len(df_cleaned.index)}")
assert len(df_cleaned.index) == data_type.clean_count

# Load
target_db.upload_to_db(target_engine, df=df_cleaned, table_name=data_type.table_name)

Processing Store Data
Store rows extracted: 451


In [20]:
extracted[extracted['store_type'] == 'NULL']

,index,address,longitude,lat,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
217,217,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
405,405,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
437,437,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [11]:
# len(df_orders['card_number'])

# df_orders['card_number'] = df_orders['card_number'].astype(str)

# df_orders.info()

import numpy as np
measurer = np.vectorize(len)

res2 = measurer(df_stores.select_dtypes(include=[object]).values.astype(str)).max(axis=0)
res2

# array([36, 36, 19, 12, 11])


array([33,  8, 20, 12, 11, 10,  2,  7, 29, 18,  8])